In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


In [27]:
class StandardLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StandardLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Defining the first LSTM layer
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)

        # Defining the second LSTM layer
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        # Fully connected layer that outputs the predicted value
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initial hidden and cell states are zero
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        # First LSTM layer
        out, (hn, cn) = self.lstm1(x, (h0, c0))

        # Second LSTM layer
        out, (hn, cn) = self.lstm2(out, (hn, cn))

        # Reshape the output from the second LSTM layer to fit the fully connected layer
        out = out[:, -1, :]  # Taking the last time step's output

        # Fully connected layer
        out = self.fc(out)
        return out

In [28]:
# ParameterCount_base
input_size = 7
hidden_size = 128
output_size = 1

In [29]:
model = StandardLSTM(input_size, hidden_size, output_size)
print(summary(model))
torch.save(model,'modelCache/LSTM_base.pth')

Layer (type:depth-idx)                   Param #
StandardLSTM                             --
├─LSTM: 1-1                              70,144
├─LSTM: 1-2                              132,096
├─Linear: 1-3                            129
Total params: 202,369
Trainable params: 202,369
Non-trainable params: 0


In [30]:
seq_len = 1  # Sequence length
batch_size = 1  # Number of sequences in the batch
x = torch.randn(batch_size, seq_len, input_size)

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model(x)

print("Input shape:", x.shape)
print("Output shape:", output.shape)
print("Output:", output)

Input shape: torch.Size([1, 1, 7])
Output shape: torch.Size([1, 1])
Output: tensor([[0.0208]])
